# bsky integration

> Fill in a module description here

In [ ]:
#| default_exp bsky

In [ ]:
#| export

import asyncio, typing, os
from atproto import AsyncClient, models
from c2i.core import generate_screenshot

In [ ]:
#| hide

from nbdev.showdoc import *

try:
  from dotenv import load_dotenv
  load_dotenv()
except: pass


In [ ]:
#| export

Notification = models.AppBskyNotificationListNotifications.Notification
FeedPost = models.AppBskyFeedPost.Record
ReplyRef = models.AppBskyFeedPost.ReplyRef

FETCH_NOTIFICATIONS_DELAY_SEC = 5

async def main(run_once = False) -> None:
    bot_handle, password = os.getenv('BSKY_BOT_HANDLE'), os.getenv('BSKY_PASSWORD')

    if not bot_handle or not password: raise ValueError('Please set BSKY_BOT_HANDLE and BSKY_PASSWORD environment variables')

    # clean up bot_handle if needed
    bot_handle = bot_handle.replace('@', '')

    async_client = AsyncClient()
    await async_client.login(bot_handle, password)

    async def on_notification_callback(notification: Notification) -> None:
        if notification.reason != "mention": return

        assert type(notification.record) is FeedPost
        post: FeedPost = notification.record

        root_post_ref = models.create_strong_ref(notification)
        reply_to = ReplyRef(parent=root_post_ref, root=root_post_ref)
        
        content = post.text.replace(f"@{bot_handle}", '').strip()
        
        with open(await generate_screenshot(code=content), 'rb') as f:
          await async_client.send_image(text='', image=f.read(), image_alt='Code snippet', reply_to=reply_to)

            

    async def listen_for_notifications(
        on_notification: typing.Callable[[Notification], typing.Coroutine[typing.Any, typing.Any, None]],
    ) -> None:
        print('Start listening for notifications...')

        while True:      
          # save the time in UTC when we fetch notifications
          last_seen_at = async_client.get_current_time_iso()

          # fetch new notifications
          response = await async_client.app.bsky.notification.list_notifications()

          # create a task list to run callbacks concurrently
          on_notification_tasks = []
          
          for notification in response.notifications:
              if not notification.is_read:
                  on_notification_tasks.append(on_notification(notification))

          # run callback on each notification
          await asyncio.gather(*on_notification_tasks)

          # mark notifications as processed (isRead=True)
          await async_client.app.bsky.notification.update_seen({'seen_at': last_seen_at})

          print('Successfully process notification. Last seen at:', last_seen_at)

          if run_once: return
            
          await asyncio.sleep(FETCH_NOTIFICATIONS_DELAY_SEC)

    # run our notification listener and register the callback on notification
    await asyncio.ensure_future(listen_for_notifications(on_notification_callback))

# Running bot on your server/computer

To run the bot, make sure your env is configured (check `.env.example`) and then use `main` function

In [ ]:
#| eval: false
await main(run_once=True)

Start listening for notifications...
Successfully process notification. Last seen at: 2024-12-06T11:29:29.457486+00:00


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()